# Mini Project - Job Salaries Dataset

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import collections
from scipy import stats
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics
from sklearn.model_selection import train_test_split
from math import sqrt
from datetime import datetime, timedelta
# Import Decision Tree Classifier model from Scikit-Learn
from sklearn.tree import DecisionTreeClassifier
# Plot the trained Decision Tree
from sklearn.tree import plot_tree
# for plotting confusion matrix
from sklearn.metrics import confusion_matrix
from collections import Counter


from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn. ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC, SVR
from sklearn.model_selection import cross_val_score, cross_validate

In [2]:
dsdata = pd.read_csv('new_ds_salaries.csv')
ds_new = dsdata.copy()

# Exploring the numeric data in hr_data
ds_num = ds_new.select_dtypes(['int64', 'float64']).copy()
ds_num.head()

,work_year,salary,salary_in_usd,remote_ratio,experience_levelN,company_sizeN,employment_typeN,employee_continentN,company_continentN
0,2020,70000,79833,0,1,2,1,0,0
1,2020,260000,260000,0,2,0,1,1,1
2,2020,85000,109024,1,2,1,1,0,0
3,2020,20000,20000,0,1,0,1,2,2
4,2020,150000,150000,1,2,2,1,2,2


## 1. Decision Tree Regressor

### 1.1 Define Model Inputs (X) & Output (y)

In [3]:
X = ds_num.drop(['salary_in_usd', 'work_year', 'salary'], axis=1)
y = ds_num['salary_in_usd']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

### 1.2 Create Decision Tree Regressor Model

In [5]:
# Create Decision Tree Model
tree_reg = tree.DecisionTreeRegressor(max_depth=40, random_state=2)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=40, random_state=2)

### 1.3 Evaluating Decision Tree Regressor Model

In [6]:
# Training MSE
train_mse = sqrt(mean_squared_error(tree_reg.predict(X_train), y_train))
print('Training Root Mean Squared Error is: ', train_mse)

# Testing mse
test_mse = sqrt(mean_squared_error(tree_reg.predict(X_test), y_test))
print('Testing Root Mean Squared Error is: ',test_mse)

Training Root Mean Squared Error is:  46043.174130143605
Testing Root Mean Squared Error is:  58347.6096446681


In [7]:
train_R2 = tree_reg.score(X_train, y_train)
print('Training R^2 value is: ', train_R2)
test_R2 = tree_reg.score(X_test, y_test)
print('Testing R^2 value is: ', test_R2)

Training R^2 value is:  0.606862069809205
Testing R^2 value is:  0.18519796612680894


### 1.4 Check using K-Fold Cross Validation to see if it will improve accuracy

In [8]:
results = cross_validate(tree_reg, X, y, scoring='neg_mean_squared_error', cv=5, return_train_score = True)
print('Training Score: ', results['train_score'])
print('Testing Score: ', results['test_score'])

Training Score:  [-1.61347460e+09 -2.07775009e+09 -2.09089211e+09 -2.10753548e+09
 -1.95745419e+09]
Testing Score:  [-5.88837912e+09 -4.47621720e+09 -2.68939630e+09 -3.07390298e+09
 -3.12131747e+09]


In [9]:
print('Cross Validation Training Root MSE is:', sqrt(sum(-results['train_score'])/len(results['train_score'])))
print('Cross Validation Testing Root MSE is:', sqrt(sum(-results['test_score'])/len(results['test_score'])))

Cross Validation Training Root MSE is: 44378.16235535028
Cross Validation Testing Root MSE is: 62047.099966221744


### 1.5 Improving Decision Tree Regressor Model

#### Using Gridsearch to improve on the model
- Finding the best Hyperparameters to select on modelling

In [10]:
param_grid = { "criterion" : ["squared_error", "friedman_mse", "absolute_error", "poisson"],
                 "min_samples_leaf" : [15,20,25,35],
                 "min_samples_split" : [16,22,26,29],
                 'max_depth' : [40,45,50,55]}
gs = GridSearchCV(tree_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv= 10, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)

-3102128028.58451
{'criterion': 'poisson', 'max_depth': 40, 'min_samples_leaf': 15, 'min_samples_split': 16}


### 1.6 Create New Decision Tree Regressor Model with new Hyperparameters

In [11]:
tree_reg_new = tree.DecisionTreeRegressor(criterion='poisson', max_depth = 40, min_samples_leaf = 15, min_samples_split=16)
tree_reg_new.fit(X_train, y_train)

DecisionTreeRegressor(criterion='poisson', max_depth=40, min_samples_leaf=15,
                      min_samples_split=16)

### 1.7 Evaluating model based on newly improved model with hyperparameters

In [12]:
#training mse
new_train_mse = sqrt(mean_squared_error(tree_reg_new.predict(X_train), y_train))
print('Training Root Mean Squared Error is: ', train_mse)
#testing mse
new_test_mse = sqrt(mean_squared_error(tree_reg_new.predict(X_test), y_test))
print('Testing Root Mean Squared Error is: ', test_mse)

Training Root Mean Squared Error is:  46043.174130143605
Testing Root Mean Squared Error is:  58347.6096446681


In [13]:
train_R2_new = tree_reg_new.score(X_train, y_train)
print('Training R^2 value is: ', train_R2_new)
test_R2_new = tree_reg_new.score(X_test, y_test)
print('Testing R^2 value is: ', test_R2_new)

Training R^2 value is:  0.3972164267578737
Testing R^2 value is:  0.50108403633547


### 1.8 Check whether using K-fold Cross Validation will further improve the accuracy on the new Decision Tree Regressor Model

In [14]:
# Evaluate the Model using K-folds Cross validation
new_results = cross_validate(tree_reg_new, X, y, scoring='neg_mean_squared_error', cv=5, return_train_score = True)
print('train_score: ', new_results['train_score'])
print('test_score: ', new_results['test_score'])

train_score:  [-2.47218656e+09 -2.90087424e+09 -2.88048434e+09 -3.08784238e+09
 -2.73538085e+09]
test_score:  [-4.17322674e+09 -3.95547985e+09 -2.77642848e+09 -2.19064834e+09
 -2.89470170e+09]


In [15]:
print('cross validation training root mean_squared_error is:', sqrt(sum(-new_results['train_score'])/len(new_results['train_score'])))
print('cross validation testing  root mean_squared_error is:', sqrt(sum(-new_results['test_score'])/len(new_results['test_score'])))

cross validation training root mean_squared_error is: 53059.90646587468
cross validation testing  root mean_squared_error is: 56551.71988361068


### 1.9 Overview of Model

In [16]:
print("**Current Decision Tree Regressor Model**")
print()
print("RMSE Scores")
print("---Before---")
print('Training Root Mean Squared Error is: ', train_mse)
print('Testing Root Mean Squared Error is: ', test_mse)
print('Cross Validation Training Root mean_squared_error is:', sqrt(sum(-results['train_score'])/len(results['train_score'])))
print('Cross Validation Testing Root mean_squared_error is:', sqrt(sum(-results['test_score'])/len(results['test_score'])))
print()
print("---After---")
print('Training Root Mean Squared Error is: ', new_train_mse)
print('Testing Root Mean Squared Error is: ', new_test_mse)
print('Cross Validation Training Root mean_squared_error is:', sqrt(sum(-new_results['train_score'])/len(new_results['train_score'])))
print('Cross Validation Testing Root mean_squared_error is:', sqrt(sum(-new_results['test_score'])/len(new_results['test_score'])))
print()
print("Accuracy")
print("---Before---")

print('training R^2 value is: ', train_R2)
print('testing R^2 value is: ', test_R2)
print()
print("---After---")
print('training R^2 value is: ', train_R2_new)
print('testing R^2 value is: ', test_R2_new)

**Current Decision Tree Regressor Model**

RMSE Scores
---Before---
Training Root Mean Squared Error is:  46043.174130143605
Testing Root Mean Squared Error is:  58347.6096446681
Cross Validation Training Root mean_squared_error is: 44378.16235535028
Cross Validation Testing Root mean_squared_error is: 62047.099966221744

---After---
Training Root Mean Squared Error is:  57012.94648250877
Testing Root Mean Squared Error is:  45657.35164886388
Cross Validation Training Root mean_squared_error is: 53059.90646587468
Cross Validation Testing Root mean_squared_error is: 56551.71988361068

Accuracy
---Before---
training R^2 value is:  0.606862069809205
testing R^2 value is:  0.18519796612680894

---After---
training R^2 value is:  0.3972164267578737
testing R^2 value is:  0.50108403633547


After applying the best hyperparameters and refitting the Decision Tree Regressor Model again

We can see that the training model and the testing model from before and after have changed drastically
The RMSE have decreased from before and after hyperparameters have been selected
However, after applying cross validation, the RMSE of training are testing models are more closely matched as compared to before.

The R^2 training and testing values have greatly improved after applying gridsearch and cross validation

However, the model can still be further improved by finding out the importance of features and dropping insignificant ones near the bottom of the tree

### 1.10 Using Feature Importance to determine important data features

In [17]:
pd.concat((pd.DataFrame(X_train.columns, columns = ['feature']), 
           pd.DataFrame(tree_reg_new.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)

,feature,importance
4,employee_continentN,0.728805
1,experience_levelN,0.193035
2,company_sizeN,0.038845
0,remote_ratio,0.019966
5,company_continentN,0.019350
3,employment_typeN,0.000000


- Thinking to drop the unimportant columns, but will be left with one or two columns to be used on the model